# Ước tính giá trị vòng đời khách hàng sử dụng mô hình chuỗi Markov

Sử dụng chuỗi Markov có thể ước tính CLV dựa trên xác suất của khách hàng chuyển tới mỗi trạng thái.

## Đọc dữ liệu

In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import inv

In [94]:
df_raw = pd.read_csv('_data/data-2.csv')
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


Trong bảng trên, chúng ta có thể thấy rằng mỗi hàng đại diện cho một mặt hàng trong một đơn đặt hàng từ một số khách hàng cụ thể. Để tính giá trị trọn đời của khách hàng, chúng ta muốn bảng ở cấp độ đơn hàng thay vì cấp sản phẩm.

## Định dạng dữ liệu ở cấp độ đơn hàng

In [100]:
def data_manipulate(df):
    """
    Thay đổi dạng và mẫu dữ liệu
    
    Trả về:
        pd.DataFrame: dataframe với một giao dịch trên một hàng
    """
    
    df['amount'] = df['Quantity']*df['UnitPrice']
    df2 = df.groupby(['CustomerID','InvoiceNo','InvoiceDate']).agg({'amount':np.sum}).reset_index().query("amount>0").reset_index(drop=True)
    df2['InvoiceDate'] = pd.to_datetime(df2['InvoiceDate'])    
    df2['date'] = df2['InvoiceDate'].apply(lambda x: x.date())  
    df2['CustomerID'] = df2['CustomerID'].apply(lambda x: str(int(x))) #.astype(int).astype(str)
    df2['month'] = df2['InvoiceDate'].apply(lambda x: x.month)
    df2 = df2.sort_values(by=['CustomerID','InvoiceDate'])
    
    return df2        

In [101]:
df = data_manipulate(df_raw)

In [102]:
df.head()

,CustomerID,InvoiceNo,InvoiceDate,amount,date,month
0,12346,541431,2011-01-18 10:01:00,77183.60,2011-01-18,1
1,12347,537626,2010-12-07 14:57:00,711.79,2010-12-07,12
2,12347,542237,2011-01-26 14:30:00,475.39,2011-01-26,1
3,12347,549222,2011-04-07 10:43:00,636.25,2011-04-07,4
4,12347,556201,2011-06-09 13:01:00,382.52,2011-06-09,6


Trong bảng, chúng ta có thể thấy rằng mỗi hàng đại diện cho một đơn đặt hàng từ một khách hàng cụ thể. Chúng ta cũng có ngày đặt hàng cũng như số tiền trên mỗi đơn hàng, đây là thông tin cần thiết khi tính giá trị vòng đời của khách hàng.

## Thống kê mô tả dữ liệu

#### Phạm vi thời gian của dữ liệu

In [7]:
df['InvoiceDate'].describe()

count                   18562
unique                  17282
top       2011-10-21 14:41:00
freq                        4
first     2010-12-01 08:26:00
last      2011-12-09 12:50:00
Name: InvoiceDate, dtype: object

#### Số lượng khách hàng trong dữ liệu

In [8]:
len(df['CustomerID'].unique())

4338

#### Số lượng khách hàng trong mỗi tháng

In [9]:
df.groupby('month').agg({'CustomerID':'nunique'}).reset_index()

,month,CustomerID
0,1,741
1,2,758
2,3,974
3,4,856
4,5,1056
5,6,991
6,7,949
7,8,935
8,9,1266
9,10,1364


#### Số lượng đơn đặt hàng của khách hàng

In [97]:
df.groupby('CustomerID').agg({'InvoiceNo':'nunique', 'amount': np.mean}).reset_index().describe()

,InvoiceNo,amount
count,4338.000000,4338.000000
mean,4.272015,418.710140
std,7.697998,1796.481888
min,1.000000,3.450000
25%,1.000000,178.450500
50%,2.000000,292.552000
75%,5.000000,428.890000
max,209.000000,84236.250000


#### Định nghĩa giai đoạn trở lại mua hàng

In [152]:
def get_purchase_period(df):
    """Số lượng ngày giữa mỗi đơn đặt hàng cho người có nhiều lần mua hàng"""
    
    # Giữ các hàng từ cid với nhiều bản ghi giao dịch
    df_multi_id = df.groupby('CustomerID').agg({'InvoiceNo':'nunique'}).reset_index().query('InvoiceNo>1')['CustomerID']    
    df_multi = df[df['CustomerID'].isin(list(df_multi_id))].copy()
    
    # số lượng ngày giữa mỗi đơn đặt hàng
    df_multi['date_prev'] = df_multi['date'].shift(1)
    df_multi['date_diff'] = df_multi['date'] - df_multi['date_prev']
    df_multi['date_diff'] = df_multi['date_diff'].apply(lambda x: x.days)
    
    # Thoát khỏi hàng đầu tiên cho mỗi khách hàng
    df_multi['cid_shift'] = df_multi['CustomerID'].shift(1)
    df_multi['is_first_row'] = df_multi['CustomerID']!=df_multi['cid_shift']
    df_multi = df_multi[df_multi['is_first_row']==False]
    
    return df_multi

In [153]:
df_return = get_purchase_period(df)

In [156]:
df_avg_return= df_return.groupby('CustomerID').agg({'date_diff':np.mean}).reset_index()

In [159]:
df_avg_return['date_diff'].describe()

count    2845.000000
mean       72.523273
std        65.463660
min         0.000000
25%        29.400000
50%        53.400000
75%        91.750000
max       366.000000
Name: date_diff, dtype: float64

Đối với những người đã thực hiện nhiều giao dịch mua, số ngày trung bình giữa mỗi đơn hàng là 73 ngày.
Số ngày trung bình giữa mỗi đơn hàng là 53 ngày.

## Xây dựng mô hình CLV

-Sử dụng recency và frequency để phân tách tất cả các khách hàng vào các trạng thái khác nhau. Đầu tiên định nghĩa một ngưỡng thời gian có thể phân chia các giao dịch vào khoảng thời gian trước và hiện tại. Sử dụng tỉ lệ mua hàng trở lại để ước tính xác suất chuyển để chúng ta sử dụng ma trận chuyển. Với giả sử xác suất không đổi cho các khoảng thời gian sau, chúng ta có thể sử dụng nó để ước tính giá trị vòng đời khách hàng. Một số giả định và các tham số cố định ta cần định nghĩa để tính giá trị cuối cùng. Một sô thông tin ước tính CLV:
-Giả định:
    +Định nghĩa ngưỡng thời gian: tháng 10/2010
    +Định nghĩa độ dài của khoảng thời gian lần mua gần nhất: 1 tháng. Recency < 1 nghĩa là khách hàng có mua hàng vào tháng 9/2010.
    +Định nghĩa các nhóm khác nhau của recency và frequency:
        R<1, F=1
        R<1, F>1
        1≤R<2, F=1
        2≤R<3, F≥1
        3≤R<4, F≥1
        4≤R, F≥1
    +Định nghĩa tỉ lệ chiết khấu: giả sử tỉ lệ chiết khấu là 10% và không đổi.
-Đầu vào của mô hình
    +Vecto ban đầu: Số lượng khách hàng trong mỗi trạng thái tại thời gian 0.
    +Ma trận chuyển: Tính ma trận chuyển theo giả định chúng ta xây dựng: xác suất chuyển là tổng số lần mua của khách hàng                         trên tổng khách hàng là xác suất chuyển cho mỗi trạng thái.
    +Vecto giá trị: Các chi phí tiếp thị tại thời điểm t là c và m là giá trị đóng góp của khách hàng, một khách hàng                              không mua tại thời gian t tạo ra -c lợi nhuận trong khi đó một khách hàng mua sinh ra m-c lợi nhuận.
             *Chi phí tiếp thị: Chi phí để có khách hàng tại mỗi thời điểm.
             *Giá trị mang lại: sử dụng tổng số tiền trung bình cho tất cả các khách hàng, giả sử rằng nếu duy trì được khách                                  hàng thì khách hàng sẽ sẽ chi số tiền này trong mỗi giai đoạn.

-Đầu ra của mô hình
    +Vecto CLV: mỗi thành phần biểu thị CLV cho các khách hàng trong mỗi giai đoạn.

---

In [13]:
def get_recency(df, snapshot_date, col_cid, col_date):
    """
    Lấy lần mua gần nhất của các khách hàng theo thời gian lần mua cuối cùng
    
   tham số truyền vào:
        df (pd.DataFrame): Dữ liệu các giao dịch
        snapshot_date (datetime.date):
            -ngưỡng thời gian được sử dụng để ước tính xác suất chuyển 
            -sử dụng ngưỡng thời gian như hôm nay (today()) trừ thời gian mua là ngưỡng thời gian trong tương lai
        col_cid (str): cột customerID    
        col_date (str): cột ngày giao dịch (InvoiceDate)
        
    Trả về:
        pd.DataFrame: một dữ liệu với cột [CustomerID, Recency]
        
        R=0 biểu thị khách hàng mua hiện tại
        R>1, ... biểu thị khách hàng không mua ở thời gian hiện tại            
    """

    # Lấy giao dịch gần đây nhất cho mỗi cid
    df_recent = df.groupby(col_cid).agg({'date':'max'}).reset_index()

    # Lấy cột recency
    df_recent['recency'] = df_recent['date'].apply(lambda x: label_recency(x, snapshot_date))
    
    return df_recent[['CustomerID', 'recency']]
    
    
def label_recency(date, snapshot_date):
    """
    match date to recency measure
    """
    
    if (date >= snapshot_date):
        recency = 'R=0' 
    elif (date >= (snapshot_date- pd.DateOffset(months=1)).date()):
        recency = 'R<1' 
    elif (date >= (snapshot_date- pd.DateOffset(months=2)).date()):
        recency = '1<=R<2'
    elif (date >= (snapshot_date- pd.DateOffset(months=3)).date()):        
        recency = '2<=R<3'  
    elif (date >= (snapshot_date- pd.DateOffset(months=4)).date()):        
        recency = '3<=R<4'         
    else:
        recency = 'R>=4'         

    return recency 
 

In [14]:
def get_frequency(df, col_cid, col_invoice):
    """
    Lấy frequency của khách hàng theo tổng số lần mua
    Tham số truyền vào:
        df (pd.DataFrame): dữ liệu giao dịch
        col_cid (str): cột mã khách hàng (CustomerID)            
        col_invoice (str): cột mã hóa đơn (InvoiceNo)

    Trả về:
        pd.DataFrame: dữ liệu với cột [CustomerID, Frequenct]
    
    """

    df_freq = df.groupby(col_cid).agg({col_invoice:'nunique'}).reset_index().rename(columns={'InvoiceNo':'TranCount'})
    df_freq['frequency'] = ['F=1' if t == 1 else 'F>1' for t in df_freq['TranCount']]
    
    return df_freq

In [64]:
def merge_date(df_freq, df_recency, col_cid = 'CustomerID'):
    """ Nối tất cả các bảng liên quan đến tính CLV với nhau và giữ một bản ghi cho một khách hàng """
    
    customer_df = pd.merge(df_freq, df_recency, on=col_cid, how='outer')
    
    # get state label
    customer_df['state'] = customer_df.apply(lambda row: label_state(row.recency, row.frequency), axis=1)
    
    return customer_df

In [17]:
def label_state(r, f):
    """Nhãn trạng thái của khách hàng"""
        
    if r=='R<1' and f=='F=1':
        state = "state 1: R<1 F=1"
    elif r=='R<1' and f=='F>1':        
        state = "state 2: R<1 F>1"    
    elif r=='1<=R<2' and f=='F=1':        
        state = "state 3: 1<=R<2 F=1"    
    elif r=='1<=R<2' and f=='F>1':        
        state = "state 4: 1<=R<2 F>1"    
    elif r=='2<=R<3':        
        state = "state 5: 2<=R<3 F>=1"    
    elif r=='3<=R<4':        
        state = "state 6: 3<=R<4 F>=1" 
    elif r=='R>=4':        
        state = "state 7: R>=4 F>=1"      

    return state

In [43]:
def count_buyer(df_initial, df_current, col_cid):
    """Cho mỗi trạng thái, số lượng người mua ở thời gian hiện tại"""
    
    df_buyer = pd.DataFrame()
    
    for s in list(df_initial['state'].unique()):
        
        df_initial_filter = df_initial[df_initial['state']==s]
        df_current_filter = df_current[df_current[col_cid].isin(df_initial_filter[col_cid])].copy()
        
        # Lấy id khách hàng riêng biệt
        unique_count = len(df_current_filter[col_cid].unique())
        
        # Lấy tổng số tiền
        total_dollar = sum(df_current_filter['amount'])
        
        df_buyer_state = pd.DataFrame({'state': [s],
                                      'BuyerCount': [unique_count],
                                      'TotalDollars': [total_dollar]})
        
        df_buyer = pd.concat([df_buyer, df_buyer_state])
        
    return df_buyer.reset_index(drop=True)

In [61]:
def get_rf(df_previous, snapshot_date, col_cid='CustomerID', col_invoice='InvoiceNo', col_date = 'date'):
    """Lấy nhóm recency và frequency cho khung dữ liệu thời gian trước"""
    
    # Tạo khung dữ liệu frequency
    df_freq = get_frequency(df_previous, 
                        col_cid, 
                        col_invoice)
    
    # Tạo khung dữ liệu recency
    df_recency = get_recency(df_previous, 
                         snapshot_date, 
                         col_cid, 
                         col_date)

    # Nối hai bảng frequency và recency, tạo nhãn trạng thái
    df_initial = merge_date(df_freq, df_recency, col_cid)
       
    return df_initial

In [60]:
def get_transition(df, snapshot_date, col_cid='CustomerID', col_invoice='InvoiceNo', col_date = 'date'):
    """Tạo khung dữ liệu với xác suất chuyển"""
    
    # Phân tách dữ liệu vào khoảng thời gian trước và hiện tại
    df_current  = df[df['date']>= snapshot_date]
    df_previous = df[df['date']< snapshot_date]
    
    #Tạo khung dữ liệu nhóm recency và frequency cho khoảng thời gian trước
    df_initial = get_rf(df_previous, 
                        snapshot_date, 
                        col_cid,
                        col_invoice, 
                        col_date)
    
    # Tạo vecto ban đầu
    df_customer = df_initial.groupby('state').agg({col_cid:'nunique'}).reset_index().rename(columns = {col_cid: 'CustomerCount'})
       
    # Cho mỗi trạng thái, đếm số lượng người mua hàng ở khoảng thời gian hiện tại
    df_buyer = count_buyer(df_initial, df_current, col_cid)
        
    # Tạo khung dữ liệu cuối cùng
    df_final = pd.merge(df_customer, df_buyer, on='state')
    df_final['BuyRate'] = df_final['BuyerCount']/df_final['CustomerCount']
    df_final['AvgDollars'] = df_final['TotalDollars']/df_final['CustomerCount']  
    
    return df_final

## Xây dựng ma trận xác suất chuyển với giá trị R, F

* **Vecto ban đầu**: Số lượng khách hàng là vecto ban đầu
* **Xác suất chuyển**: Tổng số lần mua của khách hàng trên tổng khách hàng là xác suất chuyển cho mỗi trạng thái
* **Vecto giá trị**: Sử dụng trung bình tổng số tiền tiêu dùng của mỗi khách hàng như là đóng góp của mỗi khách hàng trong mỗi trạng thái

In [70]:
snapshot_date = pd.to_datetime('2011/11/01 00:00:00').date()
df_final = get_transition(df, snapshot_date, col_cid='CustomerID', col_invoice='InvoiceNo', col_date = 'date')

In [71]:
df_final

,state,CustomerCount,BuyerCount,TotalDollars,BuyRate,AvgDollars
0,state 1: R<1 F=1,310,76,37646.22,0.245161,121.439419
1,state 2: R<1 F>1,1054,636,836151.70,0.603416,793.312808
2,state 3: 1<=R<2 F=1,210,54,20815.98,0.257143,99.123714
3,state 4: 1<=R<2 F>1,567,310,215603.56,0.546737,380.253192
4,state 5: 2<=R<3 F>=1,339,124,58426.18,0.365782,172.348614
5,state 6: 3<=R<4 F>=1,263,85,58519.57,0.323194,222.507871
6,state 7: R>=4 F>=1,1231,259,273910.77,0.210398,222.510780


In [72]:
initial_vector = np.array(df_final['CustomerCount'])
initial_vector

array([ 310, 1054,  210,  567,  339,  263, 1231])

In [73]:
value_vector = np.array(df_final['AvgDollars'])
value_vector

array([ 121.43941935,  793.31280835,   99.12371429,  380.25319224,
        172.34861357,  222.50787072,  222.51077985])

In [75]:
transition_prob = np.array(df_final['BuyRate'])
transition_prob

array([ 0.24516129,  0.60341556,  0.25714286,  0.54673721,  0.36578171,
        0.32319392,  0.21039805])

## Ma trận chuyển

In [76]:
tran_mat = np.matrix(
    ((0, transition_prob[0], 1-transition_prob[0], 0, 0, 0, 0),
     (0, transition_prob[1], 0, 1-transition_prob[1], 0, 0, 0),
     (0, transition_prob[2], 0, 0, 1-transition_prob[2], 0, 0),
     (0, transition_prob[3], 0, 0, 1-transition_prob[3], 0, 0),
     (0, transition_prob[4], 0, 0, 0, 1-transition_prob[4], 0),
     (0, transition_prob[5], 0, 0, 0, 0, 1-transition_prob[5]),
     (0, transition_prob[6], 0, 0, 0, 0, 1-transition_prob[6]))
)

tran_mat

matrix([[ 0.        ,  0.24516129,  0.75483871,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.60341556,  0.        ,  0.39658444,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.25714286,  0.        ,  0.        ,  0.74285714,
          0.        ,  0.        ],
        [ 0.        ,  0.54673721,  0.        ,  0.        ,  0.45326279,
          0.        ,  0.        ],
        [ 0.        ,  0.36578171,  0.        ,  0.        ,  0.        ,
          0.63421829,  0.        ],
        [ 0.        ,  0.32319392,  0.        ,  0.        ,  0.        ,
          0.        ,  0.67680608],
        [ 0.        ,  0.21039805,  0.        ,  0.        ,  0.        ,
          0.        ,  0.78960195]])

## Giá trị vòng đời khách hàng

Công thức xác định CLV:

$$CLV = \sum_{t=0}^{\infty} \frac{P^tv}{(1+d)^t} = \Big[ \sum_{t=0}^{\infty} \Big( \frac{P}{1+d} \Big)^t \Big] v = \Big( I - \frac{P}{1+d} \Big)^{-1} v$$

In [85]:
def get_clv(df_final, tran_mat, value_vector, d):
    """Tính CLV sử dụng công thức mô hình di chuyển và kết hơợ với tính trạng thái sử dụng hàm tính xác suất chuyển get_transition"""
    
    clv = np.dot(inv(np.identity(7) - tran_mat/(1+d)), value_vector)
    df_clv = pd.DataFrame(np.transpose(clv))
    df_clv['state'] = df_final['state']
    df_clv = df_clv.rename(columns = {0: 'CLV'})[['state', 'CLV']]
    
    return df_clv

In [88]:
df_clv1 = get_clv(df_final, tran_mat, value_vector, d=0.1)
df_clv1

,state,CLV
0,state 1: R<1 F=1,5023.981862
1,state 2: R<1 F>1,6277.199480
2,state 3: 1<=R<2 F=1,5105.554754
3,state 4: 1<=R<2 F>1,5659.615641
4,state 5: 2<=R<3 F>=1,5240.492297
5,state 6: 3<=R<4 F>=1,5169.944391
6,state 7: R>=4 F>=1,5043.434997


In [89]:
df_clv2 = get_clv(df_final, tran_mat, value_vector, d=0.05)
df_clv2

,state,CLV
0,state 1: R<1 F=1,10257.556451
1,state 2: R<1 F>1,11567.142672
2,state 3: 1<=R<2 F=1,10342.748934
3,state 4: 1<=R<2 F>1,10925.107107
4,state 5: 2<=R<3 F>=1,10474.959338
5,state 6: 3<=R<4 F>=1,10385.528354
6,state 7: R>=4 F>=1,10243.320108


Từ kết quả, chúng ta có thể thấy rằng nó thực sự có ý nghĩa về quy mô tương đối. Chẳng hạn, giá trị vòng đời của khách hàng đối với những người ở trạng thái 2 cao hơn những người ở trạng thái 1; giá trị vòng đời của khách hàng đối với những người ở trạng thái 4 cao hơn những người ở trạng thái 3 vì tỷ lệ đóng góp trung bình và tỉ lệ trở lại mua của họ cao hơn.
Cũng lưu ý rằng bằng cách sử dụng tỷ lệ chiết khấu khác nhau, thang đo tuyệt đối của kết quả CLV thực sự khác nhau rất nhiều. Điều này sẽ gây ra một số vấn đề nếu chúng ta muốn sử dụng giá trị tuyệt đối để xác định số tiền chúng ta muốn chi cho những người ở trạng thái khác nhau.